In [ ]:
# !pip3 install -U requests

In [ ]:
import plotly.graph_objs as go
import pysam
import seaborn as sns
import umap
import vcf
from plotly.offline import init_notebook_mode, iplot

In [ ]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from MulticoreTSNE import MulticoreTSNE as TSNE


from cyvcf2 import VCF, Writer

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
import urllib.request
urllib.request.urlretrieve('ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel', 'file')

In [ ]:
def vcf2df(vcf_fname):
    """Convert a subsetted vcf file to pandas DataFrame
    and return sample-level population data"""
    samples = 'ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/integrated_call_samples_v3.20130502.ALL.panel'
    dfsamples = pd.read_csv(samples, sep='\t')
    dfsamples.set_index('sample', inplace=True)
    dfsamples.drop(columns=['Unnamed: 4', 'Unnamed: 5'], inplace=True)

    vcf_file = VCF(vcf_fname)
    df = pd.DataFrame(index=vcf_file.samples)
    for variant in vcf_file():
        df[variant.ID] = variant.gt_types

    df = df.join(dfsamples, how='outer')
    df = df.drop(columns=['pop', 'super_pop', 'gender'])

    return df, dfsamples

In [ ]:
vcf_fname = 'data/Kidd.55AISNP.1kG.vcf'
df, dfsamples = vcf2df(vcf_fname)

In [ ]:
df.head(3)

In [ ]:
dfsamples.head(3)

In [ ]:
# How many samples and AISNPs?
print(df.shape)

# Are the two DataFrames the same length?
print(df.shape[0] == dfsamples.shape[0])

In [ ]:
ncols = len(df.columns)
ohe = OneHotEncoder(categories=[range(4)] * ncols, sparse=False)

X = ohe.fit_transform(df.values)

In [ ]:
def reduce_dim(X, algorithm='PCA', n_components=4):
    """Reduce the dimensionality of the 55 AISNPs
    :param X: One-hot encoded 1kG 55 AISNPs.
    :type X: array
    :param algorithm: The type of dimensionality reduction to perform. 
        One of {PCA, UMAP, TSNE}
    :type algorithm: str 
    :param n_components: The number of components to return in X_red 
    :type n_components: int
    
    :returns: The transformed X[m, n] array, reduced to X[m, n_components] by algorithm.
    """
    
    if algorithm == 'PCA':
        X_red = PCA(n_components=n_components).fit_transform(X)
    elif algorithm == 'TSNE':
        # TSNE, Barnes-Hut have dim <= 3
        if n_components > 3:
            print('The Barnes-Hut method requires the dimensionaility to be <= 3')
            return None
        else:
            X_red = TSNE(n_components=n_components, n_jobs=4).fit_transform(X)
    elif algorithm == 'UMAP':
        X_red = umap.UMAP(n_components=n_components).fit_transform(X)
    else:
        return None
    return X_red

In [ ]:
def reduce_dim_df(X, algorithm='PCA', n_components=3):
    """Reduce the dimensionality of the 55 AISNPs
    :param X: One-hot encoded 1kG 55 AISNPs.
    :type X: array
    :param X_me: Your one-hot encoded genotype array
    :type X_me: array
    :param algorithm: The type of dimensionality reduction to perform. 
        One of {PCA, UMAP, TSNE}
    :type algorithm: str 
    :param n_components: The number of components to return in X_red 
    :type n_components: int
    
    :returns: The transformed X[m, n] array - reduced to X[m, n_components] by `algorithm`
    """
    
    if algorithm == 'PCA':
        pca = PCA(n_components=n_components).fit(X)
        X_red = pca.transform(X)
#         X_red_me = pca.transform(X_me)
        # merge your data into the same table as the reference data
#         X_merged = np.vstack((X_red, X_red_me))
        X_merged=X_red
        df_red = pd.DataFrame(X_merged, 
                              columns=['component1', 'component2', 'component3'])
    elif algorithm == 'TSNE':
        # TSNE, Barnes-Hut have dim <= 3
        if n_components > 3:
            print('The Barnes-Hut method requires the dimensionaility to be <= 3')
            return None
        else:
#             X_merged = np.vstack((X, X_me))
            X_merged = X
            X_red = mTSNE(n_components=n_components, n_jobs=4).fit_transform(X_merged)
            df_red = pd.DataFrame(X_red, 
                                  columns=['component1', 'component2', 'component3'])
    elif algorithm == 'UMAP':
        umap_ = umap.UMAP(n_components=n_components).fit(X)
        X_red = umap_.transform(X)
#         X_red_me = umap_.transform(X_me) 
        # merge your data into the same table as the reference data
#         X_merged = np.vstack((X_red, X_red_me))
        X_merged=X_red
        df_red = pd.DataFrame(X_merged, 
                              columns=['component1', 'component2', 'component3'])
    else:
        return None
    
    return df_red

In [ ]:
X_emb = reduce_dim(X, algorithm='PCA', n_components=3)

In [ ]:
X_emb

In [ ]:
df_dim=reduce_dim_df(X, algorithm='PCA', n_components=3)

In [ ]:
df_dim.head(3)

In [ ]:
# df_dim = df_dim.join(dfsamples)

In [ ]:
def encode_class(pop_level='pop'):
    """Encode the population lables for plotting."""
    le = LabelEncoder()
    if pop_level == 'pop':
        labels = le.fit_transform(dfsamples['pop'].values)
    elif pop_level == 'super_pop':
        labels = le.fit_transform(dfsamples['super_pop'].values)
    else:
        return None
    return le, labels

In [ ]:
# pop_level can be either 'pop' or 'super_pop'
le, labels = encode_class(pop_level='super_pop')

In [ ]:

def plot_samples(X_emb, x_component=None, y_component=None):
    """"""
    unique = np.unique(labels)
    colors = [plt.cm.tab10_r(i/float(len(unique)-1)) for i in range(len(unique))]
    assignments = [colors[i] for i in labels]

    plt.figure(figsize=(10,10));
    for (i,cla) in enumerate(set(labels)):
        s = None
        if le.inverse_transform([cla])[0] == 'me':
            s=500
        xc = [p for (j,p) in enumerate(X_emb[:, x_component-1]) if labels[j]==cla]
        yc = [p for (j,p) in enumerate(X_emb[:, y_component-1]) if labels[j]==cla]
        cols = [c for (j,c) in enumerate(assignments) if labels[j]==cla]
        plt.scatter(xc, yc, s=s, c=cols, label=le.inverse_transform([cla])[0])
    plt.legend();
    plt.xlabel('Component {}'.format(x_component));
    plt.ylabel('Component {}'.format(y_component));
    plt.title('Projection of 1000 Genomes Samples\ninto Lower Dimensional Space\nUsing 55 AIMs from Kidd et al.');

In [ ]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
X_emb

In [ ]:
import numpy
a = df_dim[["component1","component2","component3"]].values
numpy.savetxt("comp.csv", a, delimiter=",")

In [ ]:
plot_samples(X_emb, x_component=1, y_component=2)

In [ ]:
def generate_figure_image(groups, layout):
    data = []

    for idx, val in groups:
        if idx == 'me':
            scatter = go.Scatter3d(
            name=idx,
            x=val.loc[:, 'component1'],
            y=val.loc[:, 'component2'],
            z=val.loc[:, 'component3'],
            text=[idx for _ in range(val.loc[:, 'component1'].shape[0])],
            textposition='middle right',
            mode='markers',
            marker=dict(
                size=12,
                symbol='diamond'
                )
            )
        else:
            scatter = go.Scatter3d(
                name=idx,
                x=val.loc[:, 'component1'],
                y=val.loc[:, 'component2'],
                z=val.loc[:, 'component3'],
                text=[idx for _ in range(val.loc[:, 'component1'].shape[0])],
                textposition='middle right',
                mode='markers',
                marker=dict(
                    size=4,
                    symbol='circle'
                )
            )
        data.append(scatter)

    figure = go.Figure(
        data=data,
        layout=layout
    )
    return figure

In [ ]:
layout = go.Layout(
            margin=dict(l=0, r=0, b=0, t=0),
            scene=dict(
                xaxis=dict(
                    title='Component 1',
                    showgrid=True,
                    zeroline=False,
                    showticklabels=True
                ),
                yaxis=dict(
                    title='Component 2',
                    showgrid=True,
                    zeroline=False,
                    showticklabels=True
                ),
                zaxis=dict(
                    title='Component 3',
                    showgrid=True,
                    zeroline=False,
                    showticklabels=True
                )
            )
        )

In [ ]:
df_dim